In [55]:
import pandas as pd

In [56]:
# Importamos el sales df
df_sales = pd.read_csv("sales.csv")
df_sales.shape

(2564361, 5)

In [57]:
# Importamos el flights df
df_flights = pd.read_csv("flights.csv")
df_flights.shape

(245753, 11)

In [58]:
# Convertir la columna 'STD' a formato datetime si aún no está en ese formato
df_flights['STD'] = pd.to_datetime(df_flights['STD'])

# Filtrar las filas donde el año de 'STD' no es 2024
df_filtered = df_flights[df_flights['STD'].dt.year != 2024]

# Eliminar filas donde la columna 'Aeronave' tiene valores nulos en el DataFrame filtrado
df_clean_flights = df_filtered.dropna(subset=['Aeronave'])

# Ajustar valores de 'Passengers' para que no excedan los de 'Capacity'
df_clean_flights['Passengers'] = df_clean_flights.apply(
    lambda row: min(row['Passengers'], row['Capacity']) if pd.notnull(row['Capacity']) else row['Passengers'],
    axis=1
)

<ipython-input-58-6e78ebdb1988>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_flights['Passengers'] = df_clean_flights.apply(


In [59]:
# Definir las palabras clave a buscar en 'ProductName'
palabras_clave = ['Transfer', 'Taxi', 'Combo', 'Crew', "Vivabus", "Zona", "Vivaplay", "Apto", "Akumal", "Tulum", "Promo","Hoteles","Playa", "Autobuses", "Cancun","Hsbc-Viva","Central","Cat","Hazme"]

# Crear una expresión regular que cubra todas las palabras clave
regex = '|'.join(palabras_clave)  # Esto crea un patrón 'Transfer|Taxi|Combo|Crew|...'

# Filtrar el DataFrame para excluir filas donde 'ProductName' contiene alguna de las palabras clave
# y 'TotalSales' es negativo
df_filtrado_sales = df_sales[
    ~df_sales['ProductName'].str.contains(regex, case=False, na=False) &
    (df_sales['TotalSales'] > 0)
]
df_filtrado_sales

,Flight_ID,ProductType,ProductName,Quantity,TotalSales
0,a05290288259526edd3601160b10e1de,Botanas,Carne Seca Habanero,1,55.0
1,08f6f97437df8db101b050f1110be656,Licores,Jw Red Label,2,240.0
2,4e09c949826a77207868412baeff6d30,Licores,Jack And Coke,8,576.0
3,c3e1568fe46c68d3174681d322d412b3,Licores,Jw Red Label,2,240.0
5,34ae909bf3699372fcd66f02af0bdd54,Licores,Ron Bacardi,2,240.0
...,...,...,...,...,...
2564356,638cf1904f936c50aea4a6f123ee07cf,Licores,Vino Tinto Sangre De Toro,1,155.0
2564357,04c5a19b8382a888d5c4c7b4932e783e,Licores,Vino Tinto Sangre De Toro,1,155.0
2564358,0291f6b201d2b24717a8ed7ca04f2d90,Licores,Vino Tinto Sangre De Toro,1,155.0
2564359,a10e3ac614fd434d6312897e07727c08,Licores,Vino Tinto Sangre De Toro,1,155.0


In [60]:
# Juntando escalas tomandolas como un mismo vuelo y agarrando max numero de Passengers:

columnas_adicionales = [col for col in df_clean_flights.columns if col not in ['Passengers', 'Flight_ID']]  # Asegúrate de excluir 'Passengers' y 'Flight_ID'

# Agrupar por 'Flight_ID', sumar 'Passengers' y mantener el primer valor encontrado para las otras columnas
result_flights = df_clean_flights.groupby('Flight_ID', as_index=False).agg({  # `as_index=False` mantiene 'Flight_ID' como una columna y no como índice
    'Passengers': 'max',  # Obtener el valor máximo en 'Passengers'
    **{col: 'first' for col in columnas_adicionales}  # Mantener el primer valor para las otras columnas
})

result_flights

,Flight_ID,Passengers,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Bookings
0,00004a718edba9d9ef878d08f02ae057,174.0,XA-VYD,AL,AT,Ciudad Principal,MX Amigos y Familia,2023-12-18 16:20:00,2023-12-18 18:35:00,180,106.0
1,0000cd79c0c3a9c309df6064dcacaeea,186.0,XA-VAR,AK,AD,MX Amigos y Familia,Playa,2023-10-07 18:55:00,2023-10-07 20:30:00,186,93.0
2,00010d4c88e2cb6089937fd4e4f9783b,145.0,XA-VAR,AO,BD,Playa,Ciudad Principal,2023-07-26 13:35:00,2023-07-26 14:30:00,186,81.0
3,000163f0df9cbfc35c4c06645ec512f6,173.0,XA-VBV,AW,BF,MX Amigos y Familia,Ciudad Principal,2023-10-29 06:05:00,2023-10-29 07:30:00,220,138.0
4,00017be73003a570dd426b155762769c,156.0,XA-VXC,BM,BD,Playa,Ciudad Fronteriza,2023-10-17 09:20:00,2023-10-17 13:05:00,240,95.0
...,...,...,...,...,...,...,...,...,...,...,...
119148,fffe47f84ecc55da94b2907a7317dd12,203.0,XA-VXG,BH,BM,Ciudad Fronteriza,Playa,2023-06-07 15:00:00,2023-06-07 17:15:00,240,107.0
119149,fffe4a9c19ae2320e76f02939fdff957,220.0,XA-VBN,AT,AW,Ciudad Principal,Ciudad Principal,2023-07-22 16:15:00,2023-07-22 18:00:00,220,160.0
119150,ffff138536f249f69340e0a8336f94a4,240.0,XA-VBR,AW,AK,Playa,Ciudad Principal,2023-06-11 14:00:00,2023-06-11 17:20:00,240,101.0
119151,ffff782f2ae79e385a016c00fbd994b8,181.0,XA-VAX,AT,AL,MX Amigos y Familia,Ciudad Principal,2023-11-06 13:30:00,2023-11-06 15:45:00,186,132.0


## Ruta

In [61]:
result_flights['Route'] = result_flights['DepartureStation'] + "-" + result_flights['ArrivalStation']
result_flights = result_flights.drop(["DepartureStation","Aeronave", "ArrivalStation"],axis=1)
result_flights

,Flight_ID,Passengers,Destination_Type,Origin_Type,STD,STA,Capacity,Bookings,Route
0,00004a718edba9d9ef878d08f02ae057,174.0,Ciudad Principal,MX Amigos y Familia,2023-12-18 16:20:00,2023-12-18 18:35:00,180,106.0,AL-AT
1,0000cd79c0c3a9c309df6064dcacaeea,186.0,MX Amigos y Familia,Playa,2023-10-07 18:55:00,2023-10-07 20:30:00,186,93.0,AK-AD
2,00010d4c88e2cb6089937fd4e4f9783b,145.0,Playa,Ciudad Principal,2023-07-26 13:35:00,2023-07-26 14:30:00,186,81.0,AO-BD
3,000163f0df9cbfc35c4c06645ec512f6,173.0,MX Amigos y Familia,Ciudad Principal,2023-10-29 06:05:00,2023-10-29 07:30:00,220,138.0,AW-BF
4,00017be73003a570dd426b155762769c,156.0,Playa,Ciudad Fronteriza,2023-10-17 09:20:00,2023-10-17 13:05:00,240,95.0,BM-BD
...,...,...,...,...,...,...,...,...,...
119148,fffe47f84ecc55da94b2907a7317dd12,203.0,Ciudad Fronteriza,Playa,2023-06-07 15:00:00,2023-06-07 17:15:00,240,107.0,BH-BM
119149,fffe4a9c19ae2320e76f02939fdff957,220.0,Ciudad Principal,Ciudad Principal,2023-07-22 16:15:00,2023-07-22 18:00:00,220,160.0,AT-AW
119150,ffff138536f249f69340e0a8336f94a4,240.0,Playa,Ciudad Principal,2023-06-11 14:00:00,2023-06-11 17:20:00,240,101.0,AW-AK
119151,ffff782f2ae79e385a016c00fbd994b8,181.0,MX Amigos y Familia,Ciudad Principal,2023-11-06 13:30:00,2023-11-06 15:45:00,186,132.0,AT-AL


## Duración


In [62]:
# Convertir las columnas 'STD' y 'STA' a datetime
result_flights['STD'] = pd.to_datetime(result_flights['STD'])
result_flights['STA'] = pd.to_datetime(result_flights['STA'])

# Extraer la fecha y la hora en columnas separadas
result_flights['Duration'] = result_flights['STA'] - result_flights['STD']
result_flights['Flight_Duration'] = result_flights['Duration'].dt.total_seconds() / 60
result_flights['STD_Date'] = result_flights['STD'].dt.date
result_flights['STD_Time'] = result_flights['STD'].dt.time
result_flights = result_flights.drop(["STA","STD","Duration"],axis=1)
result_flights

,Flight_ID,Passengers,Destination_Type,Origin_Type,Capacity,Bookings,Route,Flight_Duration,STD_Date,STD_Time
0,00004a718edba9d9ef878d08f02ae057,174.0,Ciudad Principal,MX Amigos y Familia,180,106.0,AL-AT,135.0,2023-12-18,16:20:00
1,0000cd79c0c3a9c309df6064dcacaeea,186.0,MX Amigos y Familia,Playa,186,93.0,AK-AD,95.0,2023-10-07,18:55:00
2,00010d4c88e2cb6089937fd4e4f9783b,145.0,Playa,Ciudad Principal,186,81.0,AO-BD,55.0,2023-07-26,13:35:00
3,000163f0df9cbfc35c4c06645ec512f6,173.0,MX Amigos y Familia,Ciudad Principal,220,138.0,AW-BF,85.0,2023-10-29,06:05:00
4,00017be73003a570dd426b155762769c,156.0,Playa,Ciudad Fronteriza,240,95.0,BM-BD,225.0,2023-10-17,09:20:00
...,...,...,...,...,...,...,...,...,...,...
119148,fffe47f84ecc55da94b2907a7317dd12,203.0,Ciudad Fronteriza,Playa,240,107.0,BH-BM,135.0,2023-06-07,15:00:00
119149,fffe4a9c19ae2320e76f02939fdff957,220.0,Ciudad Principal,Ciudad Principal,220,160.0,AT-AW,105.0,2023-07-22,16:15:00
119150,ffff138536f249f69340e0a8336f94a4,240.0,Playa,Ciudad Principal,240,101.0,AW-AK,200.0,2023-06-11,14:00:00
119151,ffff782f2ae79e385a016c00fbd994b8,181.0,MX Amigos y Familia,Ciudad Principal,186,132.0,AT-AL,135.0,2023-11-06,13:30:00


## Horario del vuelo

In [63]:
from datetime import datetime, timedelta
result_flights['STD_Time'] = pd.to_datetime(result_flights['STD_Time'], format='%H:%M:%S')

# Función para redondear a la hora más cercana
def round_hour(dt):
    if dt.minute >= 30:
        # Si los minutos son 30 o más, redondear hacia arriba
        dt += timedelta(hours=1)
    return dt.replace(minute=0, second=0)

# Aplicar la función para redondear la hora
result_flights['Hora_Salida_Redondeado'] = result_flights['STD_Time'].apply(round_hour)

# Extraer solo la hora
result_flights['Hora_Salida_Redondeado'] = result_flights['Hora_Salida_Redondeado'].dt.hour

result_flights = result_flights.drop(["STD_Time"],axis=1)
result_flights

,Flight_ID,Passengers,Destination_Type,Origin_Type,Capacity,Bookings,Route,Flight_Duration,STD_Date,Hora_Salida_Redondeado
0,00004a718edba9d9ef878d08f02ae057,174.0,Ciudad Principal,MX Amigos y Familia,180,106.0,AL-AT,135.0,2023-12-18,16
1,0000cd79c0c3a9c309df6064dcacaeea,186.0,MX Amigos y Familia,Playa,186,93.0,AK-AD,95.0,2023-10-07,19
2,00010d4c88e2cb6089937fd4e4f9783b,145.0,Playa,Ciudad Principal,186,81.0,AO-BD,55.0,2023-07-26,14
3,000163f0df9cbfc35c4c06645ec512f6,173.0,MX Amigos y Familia,Ciudad Principal,220,138.0,AW-BF,85.0,2023-10-29,6
4,00017be73003a570dd426b155762769c,156.0,Playa,Ciudad Fronteriza,240,95.0,BM-BD,225.0,2023-10-17,9
...,...,...,...,...,...,...,...,...,...,...
119148,fffe47f84ecc55da94b2907a7317dd12,203.0,Ciudad Fronteriza,Playa,240,107.0,BH-BM,135.0,2023-06-07,15
119149,fffe4a9c19ae2320e76f02939fdff957,220.0,Ciudad Principal,Ciudad Principal,220,160.0,AT-AW,105.0,2023-07-22,16
119150,ffff138536f249f69340e0a8336f94a4,240.0,Playa,Ciudad Principal,240,101.0,AW-AK,200.0,2023-06-11,14
119151,ffff782f2ae79e385a016c00fbd994b8,181.0,MX Amigos y Familia,Ciudad Principal,186,132.0,AT-AL,135.0,2023-11-06,14


In [65]:
# Asegurar que la columna STD_Date es de tipo datetime
result_flights['STD_Date'] = pd.to_datetime(result_flights['STD_Date'])

# Crear la nueva columna 'Month' extrayendo el mes de la columna 'STD_Date'
result_flights['Month'] = result_flights['STD_Date'].dt.month
result_flights = result_flights.drop(["STD_Date"],axis=1)
result_flights

,Flight_ID,Passengers,Destination_Type,Origin_Type,Capacity,Bookings,Route,Flight_Duration,Hora_Salida_Redondeado,Month
0,00004a718edba9d9ef878d08f02ae057,174.0,Ciudad Principal,MX Amigos y Familia,180,106.0,AL-AT,135.0,16,12
1,0000cd79c0c3a9c309df6064dcacaeea,186.0,MX Amigos y Familia,Playa,186,93.0,AK-AD,95.0,19,10
2,00010d4c88e2cb6089937fd4e4f9783b,145.0,Playa,Ciudad Principal,186,81.0,AO-BD,55.0,14,7
3,000163f0df9cbfc35c4c06645ec512f6,173.0,MX Amigos y Familia,Ciudad Principal,220,138.0,AW-BF,85.0,6,10
4,00017be73003a570dd426b155762769c,156.0,Playa,Ciudad Fronteriza,240,95.0,BM-BD,225.0,9,10
...,...,...,...,...,...,...,...,...,...,...
119148,fffe47f84ecc55da94b2907a7317dd12,203.0,Ciudad Fronteriza,Playa,240,107.0,BH-BM,135.0,15,6
119149,fffe4a9c19ae2320e76f02939fdff957,220.0,Ciudad Principal,Ciudad Principal,220,160.0,AT-AW,105.0,16,7
119150,ffff138536f249f69340e0a8336f94a4,240.0,Playa,Ciudad Principal,240,101.0,AW-AK,200.0,14,6
119151,ffff782f2ae79e385a016c00fbd994b8,181.0,MX Amigos y Familia,Ciudad Principal,186,132.0,AT-AL,135.0,14,11


## Preprocesando df_filtrado_sales

In [69]:
# Borrando variable ProductType
df_filtrado_sales=df_filtrado_sales.drop(["ProductType"],axis=1)
df_filtrado_sales

,Flight_ID,ProductName,Quantity,TotalSales
0,a05290288259526edd3601160b10e1de,Carne Seca Habanero,1,55.0
1,08f6f97437df8db101b050f1110be656,Jw Red Label,2,240.0
2,4e09c949826a77207868412baeff6d30,Jack And Coke,8,576.0
3,c3e1568fe46c68d3174681d322d412b3,Jw Red Label,2,240.0
5,34ae909bf3699372fcd66f02af0bdd54,Ron Bacardi,2,240.0
...,...,...,...,...
2564356,638cf1904f936c50aea4a6f123ee07cf,Vino Tinto Sangre De Toro,1,155.0
2564357,04c5a19b8382a888d5c4c7b4932e783e,Vino Tinto Sangre De Toro,1,155.0
2564358,0291f6b201d2b24717a8ed7ca04f2d90,Vino Tinto Sangre De Toro,1,155.0
2564359,a10e3ac614fd434d6312897e07727c08,Vino Tinto Sangre De Toro,1,155.0


In [72]:
pivot_df = df_filtrado_sales.pivot_table(
    index='Flight_ID',
    columns='ProductName',
    values='Quantity',
    aggfunc='sum',
    fill_value=0
)
pivot_df

ProductName,Agua Natural 600 Ml,Amstel Ultra,Arandano,Arandano Mango Mix,Arcoiris,Baileys,Baileys,Cafe 19 Cafe Clasico,Cafe 19 Capuchino,Cafe 19 Chiapas,...,Topochico Seltzer Fresa-Guayaba,Topochico Seltzer Mango,Tostitos,Tostitos Nachos Con Dip,Ultra Seltzer Frambuesa,Vino Blanco Cria Cuervos,Vino Tinto Cria Cuervos,Vino Tinto Sangre De Toro,Xx Lager,Xx Ultra
Flight_ID,,,,,,,,,,,,,,,,,,,,,
00004a718edba9d9ef878d08f02ae057,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0000cd79c0c3a9c309df6064dcacaeea,2,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,1
000163f0df9cbfc35c4c06645ec512f6,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
00017be73003a570dd426b155762769c,8,0,0,1,0,0,0,0,2,0,...,1,1,1,0,0,0,2,1,6,0
0001a43836c338f8d8650aefb11672c9,3,0,0,2,0,2,0,0,0,1,...,0,0,2,0,0,0,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffe47f84ecc55da94b2907a7317dd12,15,0,0,0,1,0,0,0,2,1,...,0,0,4,0,0,0,0,1,1,0
fffe4a9c19ae2320e76f02939fdff957,4,4,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,1,1,0
ffff138536f249f69340e0a8336f94a4,5,0,0,3,0,0,0,0,0,2,...,0,1,1,0,0,0,0,0,1,2


## Combinando datasets

In [76]:
result_df = pivot_df.merge(result_flights[["Flight_ID","Passengers",	"Destination_Type",	"Origin_Type",	"Capacity",	"Bookings",	"Route",	"Flight_Duration",	"Hora_Salida_Redondeado",	"Month"]], on='Flight_ID', how='left')
result_df.to_csv("red_data.csv")